In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
#from sklearn.cross_validation import cross_val_score
#from sklearn.grid_search import RandomizedSearchCV
from sklearn.metrics import classification_report
from seqeval.metrics import classification_report as classification_report_seqeval
import joblib
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn_crfsuite.metrics import flat_classification_report
import pickle
import gensim
from gensim.models import KeyedVectors, FastText
import gensim
from gensim.models.fasttext import load_facebook_model, load_facebook_vectors
import numpy as np
import pandas as pd
from ast import literal_eval


########################### MODEL PARAMETERS ############################
requireCauseAndEffect = True 
train_to_test_ratio = 0.9 # 10% test and 90% train
saveModelPath = "test.pkl" #"/model-causal-span/bertEmbeddings_simpleCRF.pkl"


##### DATA TO LOAD ######
dataPath = "data/cause_effect_sentences_with_IO_tags.csv"

data = pd.read_csv(dataPath, sep=";", converters={"tokenized":literal_eval, "bio_tags":literal_eval})
print(data.shape)
data.head()

(2500, 7)


,sentence,Intent,Cause,Effect,Causal association,tokenized,bio_tags
0,USER Additionally the medicines are being char...,NaN,medicines are being charged at MRP,costing much higher,1.0,"[USER, Additionally, the, medicines, are, bein...","[O, O, O, I-C, I-C, I-C, I-C, I-C, I-C, O, O, ..."
1,"I hear "" I hate being a diabetic "" .",NaN,diabetic,hate,1.0,"[I, hear, "", I, hate, being, a, diabetic, "", .]","[O, O, O, O, I-E, O, O, I-C, O, O]"
2,"i got lime for my glucose test , was n't that ...",NaN,glucose test,nauseous,1.0,"[i, got, lime, for, my, glucose, test, ,, was,...","[O, O, O, O, O, I-C, I-C, O, O, O, O, O, O, O,..."
3,Sounds like Willow 's blood sugar level is rea...,NaN,blood sugar level is real low,reduce her insulin shots,1.0,"[Sounds, like, Willow, 's, blood, sugar, level...","[O, O, O, O, I-C, I-C, I-C, I-C, I-C, I-C, O, ..."
4,USER I 've always found it too sweet mustvsay ...,NaN,dreaded diabetes,sauces are used sparingly,1.0,"[USER, I, 've, always, found, it, too, sweet, ...","[O, O, O, O, O, O, O, O, O, O, O, O, I-E, I-E,..."


In [2]:
############ Choose only sentences with both cause and effect or only sentences with either cause or effect (or both) #######


if requireCauseAndEffect: # cause and effect in sentence
    dataSentFiltered = data[(data["Cause"].notnull()) & (data["Effect"].notnull())]
else: # cause or effect or both in sentence
    dataSentFiltered = data[(data["Cause"].notnull()) | (data["Effect"].notnull())]

print(dataSentFiltered.shape)
dataSentFiltered.head()

(2118, 7)


,sentence,Intent,Cause,Effect,Causal association,tokenized,bio_tags
0,USER Additionally the medicines are being char...,NaN,medicines are being charged at MRP,costing much higher,1.0,"[USER, Additionally, the, medicines, are, bein...","[O, O, O, I-C, I-C, I-C, I-C, I-C, I-C, O, O, ..."
1,"I hear "" I hate being a diabetic "" .",NaN,diabetic,hate,1.0,"[I, hear, "", I, hate, being, a, diabetic, "", .]","[O, O, O, O, I-E, O, O, I-C, O, O]"
2,"i got lime for my glucose test , was n't that ...",NaN,glucose test,nauseous,1.0,"[i, got, lime, for, my, glucose, test, ,, was,...","[O, O, O, O, O, I-C, I-C, O, O, O, O, O, O, O,..."
3,Sounds like Willow 's blood sugar level is rea...,NaN,blood sugar level is real low,reduce her insulin shots,1.0,"[Sounds, like, Willow, 's, blood, sugar, level...","[O, O, O, O, I-C, I-C, I-C, I-C, I-C, I-C, O, ..."
4,USER I 've always found it too sweet mustvsay ...,NaN,dreaded diabetes,sauces are used sparingly,1.0,"[USER, I, 've, always, found, it, too, sweet, ...","[O, O, O, O, O, O, O, O, O, O, O, O, I-E, I-E,..."


In [3]:
from gensim.test.utils import datapath
we_path = "/home/adrian/PhD/Data/FastText_embedding_20190703/ft_wordembeddings_dim300_minCount5_URL-User-toConstant_iter10_20190703"
wordEmbeddings = FastText.load(we_path)

In [4]:
trainingDataSample = dataSentFiltered#.sample(n=500)   # Only for testing
train = trainingDataSample.sample(frac=train_to_test_ratio, random_state=0)
test = trainingDataSample.drop(train.index)
print("Train:", train.shape)
print("Test:", test.shape)
train.head()

Train: (1906, 7)
Test: (212, 7)


,sentence,Intent,Cause,Effect,Causal association,tokenized,bio_tags
1445,Benefit to having a child when diabetic : you ...,NaN,blood sugar is low,lollies,1.0,"[Benefit, to, having, a, child, when, diabetic...","[O, O, O, O, O, O, O, O, O, O, O, O, O, I-E, O..."
1476,@USER my father suffering from diabetes INSULI...,NaN,diabetes,suffering,1.0,"[@USER, my, father, suffering, from, diabetes,...","[O, O, O, I-E, O, I-C, O, O, O, O, O, O, O, O,..."
2130,I did n't knew that diabetes can be one of sid...,NaN,Thyroid disorder,diabetes,1.0,"[I, did, n't, knew, that, diabetes, can, be, o...","[O, O, O, O, O, I-E, O, O, O, O, O, O, O, I-C,..."
1320,My blood sugar has been high all evening and m...,NaN,blood sugar has been high,body hurts,1.0,"[My, blood, sugar, has, been, high, all, eveni...","[O, I-C, I-C, I-C, I-C, I-C, O, O, O, O, O, I-..."
2247,"@USER Cool , I spent the holiday talking about...",NaN,struggling,#insulin4all,1.0,"[@USER, Cool, ,, I, spent, the, holiday, talki...","[O, O, O, O, O, O, O, O, O, I-E, O, O, O, I-C, O]"


In [5]:
def get_features(word):
    word=word.lower()
    try:
         vector=wordEmbeddings[word]
    except:
        # if the word is not in vocabulary,
        # returns zeros array
        vector=np.zeros(300,)

    return vector   

def word2features(sent, i):
    word = sent[i]#[0]
    wordembedding=get_features(word)   ## word embedding vector 
#    postag = sent[i][1]
#    tag1=sent[i][2]
#    tag2=sent[i][4]
#    tag3 = sent[i][5]


    features = {
#        'bias': 1.0,
        'word.lower()': word.lower(),
#        'word[-3:]': word[-3:],
#        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
#        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
#        'postag': postag,
#        'postag[:2]': postag[:2],
#        'tag1': tag1,
#        'tag1[:2]': tag1[:2],
#        'tag2': tag2,
#        'tag2[:2]': tag2[:2],
#        'tag3': tag3,
#        'tag3[:2]': tag3[:2],
        'wordlength': len(word),
        'wordinitialcap': word[0].isupper(),
        'wordmixedcap': len([x for x in word[1:] if x.isupper()])>0,
        'wordallcap': len([x for x in word if x.isupper()])==len(word),
        'distfromsentbegin': i
    }

    # here you add 300 features (one for each vector component)
    for iv,value in enumerate(wordembedding):
        features['v{}'.format(iv)]=value
    
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, tag1, label, tag2, tag3 in sent]

def sent2tokens(sent):
    return [token for token, postag, tag1, label, tag2, tag3, tag4, tag5 in sent]

#X_train = [sent2features(s) for s in train_sents]
#y_train = [sent2labels(s) for s in train_sents]

#X_test = [sent2features(s) for s in test_sents]
#y_test = [sent2labels(s) for s in test_sents]


X_train = [sent2features(sentence) for sentence in train.tokenized.values.tolist()]
y_train = [tags for tags in train.bio_tags]

X_test = [sent2features(sentence) for sentence in test.tokenized.values.tolist()]
y_test = [tags for tags in test.bio_tags]

<ipython-input-5-82f0ae616920>:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector=wordEmbeddings[word]


In [6]:

%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False
)
crf.fit(X_train, y_train)   ### Error message when try to train


CPU times: user 23.1 s, sys: 12.7 ms, total: 23.1 s
Wall time: 23.2 s


/home/adrian/miniconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [9]:
##### save model ######
joblib.dump(crf, saveModelPath)

['test.pkl']

In [10]:
predictions = crf.predict(X_test)

In [11]:
test_true_tag = [ID for ID in np.concatenate(y_test)]#
test_predict_tag = [ID for ID in np.concatenate(predictions)]
print(classification_report(test_true_tag, test_predict_tag))

              precision    recall  f1-score   support

         I-C       0.59      0.57      0.58       495
         I-E       0.45      0.38      0.41       487
           O       0.92      0.94      0.93      4159

    accuracy                           0.85      5141
   macro avg       0.65      0.63      0.64      5141
weighted avg       0.84      0.85      0.84      5141



In [12]:
#The metrics we are seeing in this report are designed specifically for NLP tasks such as NER and POS tagging,
#in which all words of an entity need to be predicted correctly to be counted as one correct prediction. 
#Therefore, the metrics in this classification report are much lower than in scikit-learn's classification report.
#test_true_tag = [[id2tag[ID] for ID in IDS] for IDS in test_true_labels]
#test_predict_tag = [[id2tag[ID] for ID in IDS] for IDS in test_predict_labels]
print(classification_report_seqeval(y_test, predictions))

              precision    recall  f1-score   support

           C       0.56      0.54      0.55       241
           E       0.41      0.25      0.31       237

   micro avg       0.50      0.40      0.45       478
   macro avg       0.49      0.40      0.43       478
weighted avg       0.49      0.40      0.43       478



In [16]:
print(flat_classification_report(y_test, predictions))

              precision    recall  f1-score   support

         I-C       0.59      0.57      0.58       495
         I-E       0.45      0.38      0.41       487
           O       0.92      0.94      0.93      4159

    accuracy                           0.85      5141
   macro avg       0.65      0.63      0.64      5141
weighted avg       0.84      0.85      0.84      5141



In [23]:
for tokens, true_labels, predicts in zip(test.tokenized, test.bio_tags, predictions):
    print("\n")
    for token, true_label, predic in zip(tokens, true_labels, predicts):
        print(token, "true:", true_label, "predic:", predic)



I true: O predic: O
know true: O predic: O
the true: O predic: O
costs true: O predic: O
& true: O predic: O
the true: O predic: O
struggle true: I-E predic: I-E
just true: O predic: O
to true: O predic: O
get true: O predic: O
insurance true: I-C predic: I-C
to true: I-C predic: I-C
pay true: I-C predic: I-C
for true: O predic: O
supplies true: O predic: O
and true: O predic: O
insulin true: O predic: O
that true: O predic: O
are true: O predic: O
the true: O predic: O
difference true: O predic: O
between true: O predic: O
life true: O predic: O
& true: O predic: O
death true: O predic: O
. true: O predic: O


I true: O predic: O
'm true: O predic: O
noticing true: O predic: O
that true: O predic: O
covid true: I-C predic: O
really true: O predic: O
hitting true: I-E predic: O
people true: I-E predic: O
with true: I-E predic: O
diabetes true: I-E predic: I-E
hard true: I-E predic: I-E
.. true: O predic: O
I true: O predic: O
'm true: O predic: O
seeing true: O predic: O
them true: O

a true: O predic: O
a true: O predic: O
diabetic true: I-C predic: I-C
n true: O predic: O
i true: O predic: O
am true: O predic: O
not true: I-E predic: O
ashamed true: I-E predic: O
of true: O predic: O
it true: O predic: O
n true: O predic: O
i true: O predic: O
tske true: O predic: O
meds true: I-E predic: O
for true: O predic: O
it true: O predic: O


USER true: O predic: O
USER true: O predic: O
Have true: O predic: O
to true: O predic: O
pay true: I-E predic: O
for true: O predic: O
the true: O predic: O
insurance true: I-C predic: O
that true: I-C predic: O
is true: I-C predic: O
n't true: I-C predic: O
covered true: I-C predic: O
, true: O predic: O
as true: O predic: O
well true: O predic: O
as true: O predic: O
the true: O predic: O
insulin true: O predic: O
and true: O predic: O
supplies true: O predic: O
. true: O predic: O


Living true: O predic: O
w true: O predic: O
/ true: O predic: O
T1D true: I-C predic: I-C
is true: O predic: O
difficult true: O predic: O
, true: O

scared true: I-E predic: I-E
because true: O predic: O
he true: O predic: O
is true: O predic: O
also true: O predic: O
diabetic true: I-C predic: I-C
but true: O predic: O
I true: O predic: O
have true: O predic: O
faith true: O predic: O
that true: O predic: O
everything true: O predic: O
will true: O predic: O
pass true: O predic: O
! true: O predic: O


" true: O predic: O
So true: O predic: O
, true: O predic: O
although true: O predic: O
he true: O predic: O
died true: O predic: I-E
of true: O predic: O
COVID true: O predic: O
19 true: O predic: O
, true: O predic: O
he true: O predic: O
also true: O predic: O
died true: I-E predic: I-E
because true: O predic: O
he true: O predic: O
had true: O predic: O
#diabetes true: I-C predic: O
But true: O predic: O
even true: O predic: O
more true: O predic: O
proximately true: O predic: O
, true: O predic: O
he true: O predic: O
died true: O predic: I-E
because true: O predic: O
he true: O predic: O
lost true: O predic: I-E
his true: O pr

It true: O predic: O
's true: O predic: O
mad true: O predic: O
do true: O predic: O
you true: O predic: O
realise true: O predic: O
how true: O predic: O
much true: O predic: O
insulin true: I-C predic: I-C
I true: O predic: O
have true: O predic: O
to true: O predic: O
get true: O predic: O
on true: O predic: O
prescription true: O predic: O
there true: O predic: O
's true: O predic: O
no true: O predic: O
way true: O predic: O
I true: O predic: O
'd true: O predic: O
be true: O predic: O
able true: O predic: O
to true: O predic: O
afford true: O predic: O
what true: O predic: O
I true: O predic: O
needed true: O predic: O
to true: O predic: O
stay true: I-E predic: O
ALIVE true: I-E predic: O
:neutral_face: true: O predic: O
:neutral_face: true: O predic: O


USER true: O predic: O
I true: O predic: O
am true: O predic: O
considered true: O predic: O
high true: I-E predic: I-E
risk true: I-E predic: I-E
because true: O predic: O
I true: O predic: O
have true: O predic: O
type true: 

she true: O predic: O
craved true: O predic: O
: true: O predic: O
slightly_frowning_face true: O predic: O
: true: O predic: O


Just true: O predic: O
got true: O predic: O
my true: O predic: O
results true: O predic: O
for true: O predic: O
my true: O predic: O
pregnancy true: O predic: I-C
diabetes true: O predic: I-C
test true: O predic: I-C
and true: O predic: O
my true: O predic: O
glucose true: O predic: O
is true: O predic: O
within true: O predic: O
normal true: O predic: O
range true: O predic: O
: true: O predic: O
woman_dancing true: O predic: O
: true: O predic: O
I true: O predic: O
' true: O predic: O
m true: O predic: O
one true: O predic: O
happy true: I-E predic: O
mama true: O predic: O
because true: O predic: O
I true: O predic: O
do true: I-C predic: O
n't true: I-C predic: O
have true: I-C predic: O
to true: I-C predic: O
go true: I-C predic: O
the true: I-C predic: O
3 true: I-C predic: O
hr true: I-C predic: O
test true: I-C predic: O
: true: O predic: O
grinni

grandma true: O predic: O
died true: I-E predic: I-E
of true: O predic: I-E
breast true: O predic: I-E
cancer true: O predic: I-E
and true: O predic: O
diabetes true: I-C predic: I-C
, true: O predic: O
a true: O predic: O
health true: O predic: O
problem true: O predic: O
that true: O predic: O
has true: O predic: O
been true: O predic: O
sweeping true: O predic: O
this true: O predic: O
nation true: O predic: O
for true: O predic: O
decades true: O predic: O
. true: O predic: O


Although true: O predic: O
I true: O predic: O
love true: I-E predic: I-E
my true: O predic: I-E
faster true: I-C predic: I-E
acting true: I-C predic: I-E
insulin true: I-C predic: I-E
now true: O predic: O
, true: O predic: O
I true: O predic: O
used true: O predic: O
Human true: O predic: O
Insulin true: O predic: O
from true: O predic: O
when true: O predic: O
I true: O predic: O
was true: O predic: O
3 true: O predic: O
until true: O predic: O
I true: O predic: O
was true: O predic: O
in true: O predic: 

the true: O predic: O
fat true: O predic: O
people true: O predic: O
this true: O predic: O
morning true: O predic: O
that true: O predic: O
are true: O predic: O
n't true: O predic: O
afraid true: O predic: I-E
to true: O predic: I-E
take true: O predic: I-E
diabetes true: I-E predic: I-E
head true: O predic: I-E
on true: O predic: O
while true: O predic: O
waiting true: O predic: O
in true: O predic: O
the true: O predic: O
McDonald true: I-C predic: O
' true: O predic: O
s true: O predic: O
drive true: O predic: O
thru true: O predic: O
, true: O predic: O
but true: O predic: O
are true: O predic: O
scared true: O predic: O
of true: O predic: O
contracting true: O predic: O
a true: O predic: O
virus true: O predic: O
with true: O predic: O
a true: O predic: O
99 true: O predic: O
% true: O predic: O
survival true: O predic: O
rate true: O predic: O
. true: O predic: O


@USER true: O predic: O
I true: O predic: O
have true: O predic: O
no true: O predic: O
experience true: O predic:

this true: O predic: O
story true: O predic: O
breaks true: I-E predic: O
my true: I-E predic: O
heart true: I-E predic: O
. true: O predic: O


@USER true: O predic: O
Now true: O predic: O
I true: O predic: O
eat true: I-E predic: I-E
low true: I-E predic: I-E
carb true: I-E predic: I-E
b true: O predic: O
/ true: O predic: O
c true: O predic: O
I true: O predic: O
have true: O predic: O
to true: O predic: O
in true: O predic: O
order true: O predic: O
to true: O predic: O
keep true: O predic: O
my true: O predic: O
BGs true: I-C predic: I-E
down true: I-C predic: I-E
. true: O predic: O


@USER true: O predic: O
Since true: O predic: O
beating true: O predic: O
my true: O predic: O
Type true: I-C predic: I-C
2 true: I-C predic: I-C
Diabetes true: I-C predic: I-C
with true: O predic: I-C
exercise true: I-E predic: I-C
and true: O predic: O
using true: O predic: O
the true: O predic: O
5 true: O predic: O
Star true: O predic: O
rating true: I-E predic: O
on true: I-E predic: O
foods t

In [ ]:

# BENCHMARK: 
              precision    recall  f1-score   support

         I-C       0.59      0.57      0.58       495
         I-E       0.45      0.38      0.41       487
           O       0.92      0.94      0.93      4159

    accuracy                           0.85      5141
   macro avg       0.65      0.63      0.64      5141
weighted avg       0.84      0.85      0.84      5141

              precision    recall  f1-score   support

           C       0.56      0.54      0.55       241
           E       0.41      0.25      0.31       237

   micro avg       0.50      0.40      0.45       478
   macro avg       0.49      0.40      0.43       478
weighted avg       0.49      0.40      0.43       478



In [17]:
aa = joblib.load(saveModelPath)

In [18]:
aa

/home/adrian/miniconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/base.py:209: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)